In [1]:
import pandas as pd
import numpy as np
import re
from matplotlib import pyplot
import scipy.fftpack 
from os import walk
from sklearn.decomposition import PCA
from packages import utils
from scipy.signal import butter, lfilter,freqz
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [2]:
def split_df_into_channels(df):
    df=df.drop(['Elements'], axis=1)
    df=df.dropna()
    dfs=np.split(df,[21,25],axis=1)
    timeStamp=dfs[0]['TimeStamp']
    dfs[1]=dfs[1].join(timeStamp)
    df_channels=dfs[1]
    df_channels=df_channels[["TimeStamp","RAW_TP9","RAW_AF7","RAW_AF8","RAW_TP10"]]
    raw_tp9=df_channels['RAW_TP9']
    raw_tp10=df_channels['RAW_TP10']
    raw_af7=df_channels['RAW_AF7']
    raw_af8=df_channels['RAW_AF8']
    raw_tp9=raw_tp9[1280:]
    raw_tp10=raw_tp10[1280:]
    raw_af7=raw_af7[1280:]
    raw_af8=raw_af8[1280:]

    return raw_tp9,raw_tp10,raw_af7,raw_af8

In [3]:
def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [4]:
def compress_channel(channel):
    compressed=pd.Series()
    lower_bound=0
    for i in range(1,1201):
        upper_bound=64*i
        values=channel[lower_bound:upper_bound]
#         print("NEXT")
#         print("LOWER {}" .format(lower_bound))
#         print("UPPER {}" .format(upper_bound))
        lower_bound=upper_bound
        values=np.mean(values)
        vals=pd.Series(values)
        compressed=compressed.append(vals,ignore_index=True)
    return compressed

In [5]:
def get_compressed_channel(raw_tp9,raw_tp10,raw_af7,raw_af8):
    raw_tp9_compressed=compress_channel(raw_tp9)
    raw_tp10_compressed=compress_channel(raw_tp10)
    raw_af7_compressed=compress_channel(raw_af7)
    raw_af8_compressed=compress_channel(raw_af8)
    
    return raw_tp9_compressed,raw_tp10_compressed,raw_af7_compressed,raw_af8_compressed


In [6]:
def split_into_labels(compressed_channel):
    rest=[]
    A=[]
    B=[]

    rest.append(compressed_channel[0:40])
    A.append(compressed_channel[40:60])
    rest.append(compressed_channel[60:100])
    B.append(compressed_channel[100:120])

    rest.append(compressed_channel[120:160])
    A.append(compressed_channel[160:180])
    rest.append(compressed_channel[180:220])
    B.append(compressed_channel[220:240])

    rest.append(compressed_channel[240:280])
    A.append(compressed_channel[280:300])
    rest.append(compressed_channel[300:340])
    B.append(compressed_channel[340:360])

    rest.append(compressed_channel[360:400])
    A.append(compressed_channel[400:420])
    rest.append(compressed_channel[420:460])
    B.append(compressed_channel[460:480])

    rest.append(compressed_channel[480:520])
    A.append(compressed_channel[520:540])
    rest.append(compressed_channel[540:580])
    B.append(compressed_channel[580:600])

    rest.append(compressed_channel[600:640])
    A.append(compressed_channel[640:660])
    rest.append(compressed_channel[660:700])
    B.append(compressed_channel[700:720])

    rest.append(compressed_channel[720:760])
    A.append(compressed_channel[760:780])
    rest.append(compressed_channel[780:820])
    B.append(compressed_channel[820:840])

    rest.append(compressed_channel[840:880])
    A.append(compressed_channel[880:900])
    rest.append(compressed_channel[900:940])
    B.append(compressed_channel[940:960])

    rest.append(compressed_channel[960:1000])
    A.append(compressed_channel[1000:1020])
    rest.append(compressed_channel[1020:1060])
    B.append(compressed_channel[1060:1080])

    rest.append(compressed_channel[1080:1120])
    A.append(compressed_channel[1120:1140])
    rest.append(compressed_channel[1140:1180])
    B.append(compressed_channel[1180:1200])
    
    return A,B,rest


In [7]:
def get_labels():
    tp9_A,tp9_B,tp9_rest=split_into_labels(raw_tp9_compressed)
    tp10_A,tp10_B,tp10_rest=split_into_labels(raw_tp10_compressed)
    af7_A,af7_B,af7_rest=split_into_labels(raw_af7_compressed)
    af8_A,af8_B,af8_rest=split_into_labels(raw_af8_compressed)

In [8]:
def labeled_channel_to_np(labeled_channel):
    labeled_channel_np=[]
    for val,i in enumerate(labeled_channel):
        labeled_channel_np.append(i.to_numpy())
    return labeled_channel_np

In [9]:
def convert_to_single_row(tp9,tp10,af7,af8,group):
    if group=='A':
        label=[0,1,0]
    elif group=='B':
        label=[0,0,1]
    else:
        label=[1,0,0]
    master=[]
    for val,i in enumerate(tp9):
        for index,j in enumerate(i):
            single_row=[]
            single_row.append(tp9[val][index])
            single_row.append(tp10[val][index])
            single_row.append(af7[val][index])
            single_row.append(af8[val][index])
            single_row.append(label)

            master.append(single_row)
    for i in range(0,3):
        single_row=[]
        single_row.append(tp9[val][index])
        single_row.append(tp10[val][index])
        single_row.append(af7[val][index])
        single_row.append(af8[val][index])
        single_row.append(label)
        master.append(single_row)
    return master

In [10]:
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
    # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
    # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
#         seq_x = sequences[i:end_ix, :-1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X),np.array(y)

In [11]:
def apply_pca(labelled_X):
    X_a_pca=[]
    pca=PCA(n_components=4)
    for val,i in enumerate(labelled_X):
    #     if val>2:
    #         break
#         pca.fit(i)
#         X_a_pca.append(pca.singular_values_)
        X_a_pca.append(pca.fit_transform(i))
    return X_a_pca

In [12]:
def assemble_dataset(df):
    raw_tp9,raw_tp10,raw_af7,raw_af8=split_df_into_channels(df)
    filtered_tp9=butter_lowpass_filter(raw_tp9,30,256,6)
    filtered_tp10=butter_lowpass_filter(raw_tp10,30,256,6)
    filtered_af7=butter_lowpass_filter(raw_af7,30,256,6)
    filtered_af8=butter_lowpass_filter(raw_af8,30,256,6)
    filtered_tp9_compressed,filtered_tp10_compressed,filtered_af7_compressed,filtered_af8_compressed=get_compressed_channel(
                                                                                                        filtered_tp9,
                                                                                                        filtered_tp10,
                                                                                                        filtered_af7,
                                                                                                        filtered_af8)
    
    tp9_A,tp9_B,tp9_rest=split_into_labels(filtered_tp9_compressed)
    tp10_A,tp10_B,tp10_rest=split_into_labels(filtered_tp10_compressed)
    af7_A,af7_B,af7_rest=split_into_labels(filtered_af7_compressed)
    af8_A,af8_B,af8_rest=split_into_labels(filtered_af8_compressed)
    
    tp9_A_np=labeled_channel_to_np(tp9_A)
    tp10_A_np=labeled_channel_to_np(tp10_A)
    af7_A_np=labeled_channel_to_np(af7_A)
    af8_A_np=labeled_channel_to_np(af8_A)

    tp9_B_np=labeled_channel_to_np(tp9_B)
    tp10_B_np=labeled_channel_to_np(tp10_B)
    af7_B_np=labeled_channel_to_np(af7_B)
    af8_B_np=labeled_channel_to_np(af8_B)

    tp9_rest_np=labeled_channel_to_np(tp9_rest)
    tp10_rest_np=labeled_channel_to_np(tp10_rest)
    af7_rest_np=labeled_channel_to_np(af7_rest)
    af8_rest_np=labeled_channel_to_np(af8_rest)
    
    master_A_np=np.array(convert_to_single_row(tp9_A_np,tp10_A_np,af7_A_np,af8_A_np,group='A'))
    master_B_np=np.array(convert_to_single_row(tp9_B_np,tp10_B_np,af7_B_np,af8_B_np,group='B'))
    master_rest_np=np.array(convert_to_single_row(tp9_rest_np,tp10_rest_np,af7_rest_np,af8_rest_np,group='rest'))
    
    
   
    X_a,Y_a=split_sequences(master_A_np,4)
    X_b,Y_b=split_sequences(master_B_np,4)
    X_rest,Y_rest=split_sequences(master_rest_np,4)
    
    X_a_pca=apply_pca(X_a)
    X_b_pca=apply_pca(X_b)
    X_rest_pca=apply_pca(X_rest)
    X_a_pca_np=np.array(X_a_pca)
    X_b_pca_np=np.array(X_b_pca)
    X_rest_pca_np=np.array(X_rest_pca)
    X_a=X_a_pca_np
    X_b=X_b_pca_np
    X_rest=X_rest_pca_np

#     return master_A_np,Y_a,master_B_np,Y_b,master_rest_np,Y_rest
    return X_a,Y_a,X_b,Y_b,X_rest,Y_rest

In [13]:
def create_masters(X_a,Y_a,X_b,Y_b,X_rest,Y_rest):
    rest_counter=0
    A_counter=0
    B_counter=0

    X_master=[]
    Y_master=[]
    for i in range(1,6):
        X_master.append(X_rest[(rest_counter)*20:(1+rest_counter)*20])
        Y_master.append(Y_rest[(rest_counter)*20:(1+rest_counter)*20])
        X_master.append(X_rest[(1+rest_counter)*20:(2+rest_counter)*20])
        Y_master.append(Y_rest[(1+rest_counter)*20:(2+rest_counter)*20])

        X_master.append(X_a[(A_counter)*20:(1+A_counter)*20])
        Y_master.append(Y_a[(A_counter)*20:(1+A_counter)*20])

        X_master.append(X_rest[(2+rest_counter)*20 : (3+rest_counter)*20])
        Y_master.append(Y_rest[(2+rest_counter)*20:(3+rest_counter)*20])
        X_master.append(X_rest[(3+rest_counter)*20 : (4+rest_counter)*20])
        Y_master.append(Y_rest[(3+rest_counter)*20:(4+rest_counter)*20])

        X_master.append(X_b[(B_counter)*20:(1+B_counter)*20])
        Y_master.append(Y_b[(B_counter)*20:(1+B_counter)*20])

        rest_counter=rest_counter+4
        A_counter=A_counter+1
        B_counter=B_counter+1
        
    X_master_np=np.array(X_master)
    Y_master_np=np.array(Y_master)
    return X_master_np,Y_master_np

In [14]:
files=[]
path=r'C:\\Users\\harri\\Desktop\\CorTech\\Hand_ErpDetection\\Recordings_2\\CSV_Files'
for (dirpath, dirnames, filenames) in walk(path):
    print(dirpath)
    files.extend(filenames)
    break

C:\\Users\\harri\\Desktop\\CorTech\\Hand_ErpDetection\\Recordings_2\\CSV_Files


In [15]:
print(files)

['mindMonitor_2020-06-21--15-02-35.csv', 'mindMonitor_2020-06-22--11-39-14.csv', 'mindMonitor_2020-06-27--18-16-24.csv', 'mindMonitor_2020-06-27--18-31-15.csv', 'mindMonitor_2020-06-27--18-41-43.csv', 'mindMonitor_2020-06-27--18-56-16.csv', 'mindMonitor_2020-06-27--19-07-40.csv', 'mindMonitor_2020-06-27--19-14-36.csv', 'mindMonitor_2020-06-27--19-22-46.csv']


In [16]:
X_master=[]
Y_master=[]
for file_names in files:
    file_dir=path+r'\\'+file_names
    print(file_dir)
    df=pd.read_csv(file_dir)
# master_A_np,Y_a,master_B_np,Y_b,master_rest_np,Y_rest=assemble_dataset(df)

    X_a,Y_a,X_b,Y_b,X_rest,Y_rest=assemble_dataset(df)
    X_all,Y_all=create_masters(X_a,Y_a,X_b,Y_b,X_rest,Y_rest)
    X_master.append(X_all)
    Y_master.append(Y_all)

C:\\Users\\harri\\Desktop\\CorTech\\Hand_ErpDetection\\Recordings_2\\CSV_Files\\mindMonitor_2020-06-21--15-02-35.csv


C:\Users\harri\Anaconda3\lib\site-packages\sklearn\decomposition\_pca.py:456: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var
C:\Users\harri\Anaconda3\lib\site-packages\sklearn\decomposition\_pca.py:456: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var
C:\Users\harri\Anaconda3\lib\site-packages\sklearn\decomposition\_pca.py:456: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var


C:\\Users\\harri\\Desktop\\CorTech\\Hand_ErpDetection\\Recordings_2\\CSV_Files\\mindMonitor_2020-06-22--11-39-14.csv


C:\Users\harri\Anaconda3\lib\site-packages\sklearn\decomposition\_pca.py:456: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var
C:\Users\harri\Anaconda3\lib\site-packages\sklearn\decomposition\_pca.py:456: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var
C:\Users\harri\Anaconda3\lib\site-packages\sklearn\decomposition\_pca.py:456: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var


C:\\Users\\harri\\Desktop\\CorTech\\Hand_ErpDetection\\Recordings_2\\CSV_Files\\mindMonitor_2020-06-27--18-16-24.csv


...

C:\Users\harri\Anaconda3\lib\site-packages\sklearn\decomposition\_pca.py:456: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var
C:\Users\harri\Anaconda3\lib\site-packages\sklearn\decomposition\_pca.py:456: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var
C:\Users\harri\Anaconda3\lib\site-packages\sklearn\decomposition\_pca.py:456: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var


C:\\Users\\harri\\Desktop\\CorTech\\Hand_ErpDetection\\Recordings_2\\CSV_Files\\mindMonitor_2020-06-27--19-14-36.csv


C:\Users\harri\Anaconda3\lib\site-packages\sklearn\decomposition\_pca.py:456: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var
C:\Users\harri\Anaconda3\lib\site-packages\sklearn\decomposition\_pca.py:456: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var
C:\Users\harri\Anaconda3\lib\site-packages\sklearn\decomposition\_pca.py:456: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var


C:\\Users\\harri\\Desktop\\CorTech\\Hand_ErpDetection\\Recordings_2\\CSV_Files\\mindMonitor_2020-06-27--19-22-46.csv


C:\Users\harri\Anaconda3\lib\site-packages\sklearn\decomposition\_pca.py:456: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var
C:\Users\harri\Anaconda3\lib\site-packages\sklearn\decomposition\_pca.py:456: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var
C:\Users\harri\Anaconda3\lib\site-packages\sklearn\decomposition\_pca.py:456: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var


In [17]:
X_master_np=np.array(X_master)
Y_master_np=np.array(Y_master)
X_final=np.reshape(X_master_np,((X_master_np.shape[0] *600),4,4))
Y_final=np.reshape(Y_master_np,((Y_master_np.shape[0] *600),3))
print(X_final.shape)
print(Y_final.shape)

(5400, 4, 4)
(5400, 3)


Assemble training, testing data and model

In [18]:
 X_train, X_test, Y_train, Y_test = train_test_split(X_final, Y_final, test_size=0.2, random_state=1)

In [19]:
model=Sequential()
model.add(LSTM(25,activation='relu',input_shape=(4,4)))
model.add(Dense(3,activation='sigmoid'))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [20]:
model.fit(X_train,Y_train,batch_size=60,epochs=20,verbose=2, validation_split=0.15,validation_steps=30,
          use_multiprocessing=False)

Train on 3672 samples, validate on 648 samples
Epoch 1/20
3672/3672 - 4s - loss: 1.1161 - accuracy: 0.4681 - val_loss: 2.6489 - val_accuracy: 0.6385
Epoch 2/20
3672/3672 - 0s - loss: 0.9160 - accuracy: 0.6699 - val_loss: 2.4568 - val_accuracy: 0.6588
Epoch 3/20
3672/3672 - 0s - loss: 0.8488 - accuracy: 0.6721 - val_loss: 2.3693 - val_accuracy: 0.6616
Epoch 4/20
3672/3672 - 0s - loss: 0.8187 - accuracy: 0.6721 - val_loss: 2.3361 - val_accuracy: 0.6633
Epoch 5/20
3672/3672 - 0s - loss: 0.8033 - accuracy: 0.6724 - val_loss: 2.3283 - val_accuracy: 0.6667
Epoch 6/20
3672/3672 - 0s - loss: 0.7965 - accuracy: 0.6729 - val_loss: 2.3105 - val_accuracy: 0.6661
Epoch 7/20
3672/3672 - 0s - loss: 0.7901 - accuracy: 0.6740 - val_loss: 2.3085 - val_accuracy: 0.6672
Epoch 8/20
3672/3672 - 0s - loss: 0.7870 - accuracy: 0.6743 - val_loss: 2.3129 - val_accuracy: 0.6672
Epoch 9/20
3672/3672 - 0s - loss: 0.7833 - accuracy: 0.6746 - val_loss: 2.3160 - val_accuracy: 0.6644
Epoch 10/20
3672/3672 - 0s - loss: 

In [21]:
predictions=model.predict(X_test,verbose=2)
test_predictions=np.argmax(predictions,axis=1)
test_Y_pred=np.argmax(Y_test,axis=1)

1080/1 - 0s


In [22]:
print(classification_report(test_Y_pred,test_predictions))

              precision    recall  f1-score   support

           0       0.67      0.98      0.79       708
           1       0.35      0.04      0.07       189
           2       0.29      0.04      0.07       183

    accuracy                           0.65      1080
   macro avg       0.44      0.35      0.31      1080
weighted avg       0.55      0.65      0.54      1080

